#Tell Me a Story
Children's story generator using a [GPT-2](https://openai.com/blog/better-language-models/) network fine-tuned on children's stories from the Guttenberg project (via [bAbI](https://research.fb.com/downloads/babi/)). This notebook will only work in full on [colab](colab.research.google.com), as it saves the resultant model to the user's google drive.

In [0]:
! pip install transformers

     |████████████████████████████████| 317kB 2.8MB/s 
     |████████████████████████████████| 645kB 53.1MB/s 
     |████████████████████████████████| 860kB 47.0MB/s 
     |████████████████████████████████| 1.0MB 41.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=506f1c42c68372e6244e2e51b928d71ad9b8efb6fddda26e229d7760033d652a
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
! rm -rf transformers
! git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 12628 (delta 133), reused 154 (delta 104), pack-reused 12407
Receiving objects: 100% (12628/12628), 6.63 MiB | 5.71 MiB/s, done.
Resolving deltas: 100% (9221/9221), done.
Note: checking out '3ddce1d74cda5be47704381e657ee22ce5a5fc7b'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [0]:
# get the data
import re
from requests import get

url = "http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz"
fn = re.search("[^/]+$", url).group(0)
response = get(url)
with open(fn, "wb") as f:
  f.write(response.content)

! tar xfz {fn} 2> /dev/null

In [0]:
# munge it
from pathlib import PosixPath
from os import makedirs

data_path = PosixPath("/content/CBTest/data")
makedirs(data_path/"train", exist_ok=True)
makedirs(data_path/"valid", exist_ok=True)
cbt_train_file = data_path/"train/cbt_train_cleaned.txt"
cbt_valid_file = data_path/"valid/cbt_valid_cleaned.txt"
# we don't care about test, so add it to the train set
! echo "Cleaning training data"
! cat {data_path}/cbt_train.txt {data_path}/cbt_test.txt | tqdm | grep -v _BOOK_TITLE | \
perl -pe 's/-L[CS]B-.*?-R[CS]B-//g; s/-L[CS]B-.*$// if ! /-R[CS]B-/; s/^.*-R[CS]B-// if ! /-L[CS]B-/; s/-LRB-/(/g; s/-RRB-/)/g;' \
> {cbt_train_file}
! echo "Cleaning validation data"
! grep -v _BOOK_TITLE {data_path}/cbt_valid.txt | tqdm | \
perl -pe 's/-L[CS]B-.*?-R[CS]B-//g; s/-L[CS]B-.*$// if ! /-R[CS]B-/; s/^.*-R[CS]B-// if ! /-L[CS]B-/; s/-LRB-/(/g; s/-RRB-/)/g;' \
 > {cbt_valid_file}

Cleaning training data
280165it [00:00, 494072.12it/s]
Cleaning validation data
12742it [00:00, 408201.81it/s]


In [0]:
! python transformers/examples/run_lm_finetuning.py \
    --output_dir=model \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train \
    --train_data_file={cbt_train_file} \
    --do_eval \
    --eval_data_file={cbt_valid_file} \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2

11/26/2019 18:44:03 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/26/2019 18:44:04 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json not found in cache or force_download set to True, downloading to /tmp/tmpcb9b66ei
100% 176/176 [00:00<00:00, 135822.91B/s]
11/26/2019 18:44:04 - INFO - transformers.file_utils -   copying /tmp/tmpcb9b66ei to cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.085d5f6a8e7812ea05ff0e6ed0645ab2e75d80387ad55c1ad9806ee70d272f80
11/26/2019 18:44:04 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.085d5f6a8e7812ea05ff0e6ed0645ab2e75d80387ad55c1ad9806ee70d272f80
11/26/2019 18:44:04 - INFO - transformers.file_utils -   removing temp file /tmp/tmpcb9b66ei
11/26/2019 18:44

In [0]:
import random

seed = random.randint(1,50000)
length = 500
prompt = "Once upon a time, Alice fell down a well, where she met a very curious rabbit."
! python transformers/examples/run_generation.py \
  --model_type=gpt2 \
  --model_name_or_path="./model" \
  --prompt="{prompt}" \
  --length={length} \
  --seed={seed}

11/26/2019 20:51:03 - INFO - transformers.tokenization_utils -   Model name './model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, distilgpt2). Assuming './model' is a path or url to a directory containing tokenizer files.
11/26/2019 20:51:03 - INFO - transformers.tokenization_utils -   loading file ./model/vocab.json
11/26/2019 20:51:03 - INFO - transformers.tokenization_utils -   loading file ./model/merges.txt
11/26/2019 20:51:03 - INFO - transformers.tokenization_utils -   loading file ./model/added_tokens.json
11/26/2019 20:51:03 - INFO - transformers.tokenization_utils -   loading file ./model/special_tokens_map.json
11/26/2019 20:51:03 - INFO - transformers.tokenization_utils -   loading file ./model/tokenizer_config.json
11/26/2019 20:51:03 - INFO - transformers.configuration_utils -   loading configuration file ./model/config.json
11/26/2019 20:51:03 - INFO - transformers.configuration_utils -   Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
drive_dir = "/content/drive/My Drive/Colab Notebooks/models/childrens-stories_fine-tuned_gpt2"
! rm -rf "{drive_dir}"
! mkdir "{drive_dir}"
! cp /content/model/*.json /content/model/*.txt /content/model/*.bin "{drive_dir}"

#Inference from Drive

In [0]:
drive_dir = "/content/drive/My Drive/Colab Notebooks/models/childrens-stories_fine-tuned_gpt2"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! pip install transformers

     |████████████████████████████████| 317kB 3.5MB/s 
     |████████████████████████████████| 860kB 42.5MB/s 
     |████████████████████████████████| 1.0MB 31.2MB/s 
     |████████████████████████████████| 645kB 36.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=d03a8e5ea483a34aa58360ec46602cb28295e1a4a9fc8445fb1ceb979353d671
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
! rm -rf transformers
# enhanced to use past with GPT-2
! git clone https://github.com/thisisrandy/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 12479 (delta 38), reused 52 (delta 30), pack-reused 12409
Receiving objects: 100% (12479/12479), 6.56 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (9130/9130), done.


In [0]:
import random

seed = random.randint(1,50000)
length = 1000
prompt = "Once upon a time, Alice fell down a well, where she met a very curious rabbit."
! python transformers/examples/run_generation.py \
  --model_type=gpt2 \
  --model_name_or_path="{drive_dir}" \
  --prompt="{prompt}" \
  --length={length} \
  --seed={seed}

11/24/2019 17:57:02 - INFO - transformers.tokenization_utils -   Model name '/content/drive/My Drive/Colab Notebooks/models/childrens-stories_fine-tuned_gpt2_1000' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, distilgpt2). Assuming '/content/drive/My Drive/Colab Notebooks/models/childrens-stories_fine-tuned_gpt2_1000' is a path or url to a directory containing tokenizer files.
11/24/2019 17:57:03 - INFO - transformers.tokenization_utils -   loading file /content/drive/My Drive/Colab Notebooks/models/childrens-stories_fine-tuned_gpt2_1000/vocab.json
11/24/2019 17:57:03 - INFO - transformers.tokenization_utils -   loading file /content/drive/My Drive/Colab Notebooks/models/childrens-stories_fine-tuned_gpt2_1000/merges.txt
11/24/2019 17:57:03 - INFO - transformers.tokenization_utils -   loading file /content/drive/My Drive/Colab Notebooks/models/childrens-stories_fine-tuned_gpt2_1000/added_tokens.json
11/24/2019 17:57:03 - INFO - transformers.tokenization_utils -  